In [2]:
from gmail_api import fetch_spam_and_ham

spam_mails, ham_mails = fetch_spam_and_ham(100)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=628982370333-kkqj66e4850ij5qm0nr2l4b6a514s23e.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=uZEvLpvEySiby86NC1MGG9cj5gUsfJ&access_type=offline
Found 17 spam messages and 100 ham messages.


In [3]:
p_spam = len(spam_mails)/(len(spam_mails) + len(ham_mails))
p_ham = len(ham_mails)/(len(spam_mails) + len(ham_mails))

In [4]:
spam_iterator = list(spam_mails.items())
ham_iterator = list(ham_mails.items())

In [5]:
spam_iterator[1][1]

{'subject': 'Your Swiggy Instamart order was successfully delivered',
 'sender': 'noreply@swiggy.in',
 'date': 'Thu, 19 Jun 2025 11:52:46 +0000',
 'content': ''}

In [6]:
spam_word_dict = {}
ham_word_dict = {}

In [7]:
for item in spam_iterator:
    _, metadata_dict = item
    content = metadata_dict['content']
    for word in content.split():
        word = word.lower()
        if word not in spam_word_dict:
            spam_word_dict[word] = 0
        spam_word_dict[word] += 1
for item in ham_iterator:
    _, metadata_dict = item
    content = metadata_dict['content']
    for word in content.split():
        word = word.lower()
        if word not in ham_word_dict:
            ham_word_dict[word] = 0
        ham_word_dict[word] += 1

In [8]:
spam_word_count = sorted(spam_word_dict.items(), key=lambda x: x[1], reverse=True)
#use new_dict = sorted(dict.items()) alone in case you want to sort by key
ham_word_count = sorted(ham_word_dict.items(), key=lambda x: x[1], reverse=True)

In [9]:
total_words_in_spam = sum(spam_word_dict.values())
total_words_in_ham = sum(ham_word_dict.values())
vocab_size = len(spam_word_dict) + len(ham_word_dict)

In [10]:
laplace_smoothing_factor = 1

In [11]:
spam_word_frequency = []
ham_word_frequency = []
# we are creating a new list of tuples with the word, count, and frequency, because we cannot modify the 
for i in range(len(spam_word_count)):
    word, count = spam_word_count[i]
    word_frequency = (count + laplace_smoothing_factor) / (total_words_in_spam + laplace_smoothing_factor * vocab_size)
    spam_word_frequency.append((spam_word_count[i][0], count, word_frequency))

for i in range(len(ham_word_count)):
    word, count = ham_word_count[i]
    word_frequency = (count + laplace_smoothing_factor) / (total_words_in_ham + laplace_smoothing_factor * vocab_size)
    ham_word_frequency.append((ham_word_count[i][0], count, word_frequency))

In [12]:
# We can also overwrite the original list of tuples with another value, if we want to keep the original list intact
spam_word_count = [(word, count, (count + laplace_smoothing_factor) / (total_words_in_spam + laplace_smoothing_factor * vocab_size)) for word, count in spam_word_count]
ham_word_count = [(word, count, (count + laplace_smoothing_factor) / (total_words_in_ham + laplace_smoothing_factor * vocab_size)) for word, count in ham_word_count]

In [1]:
# Word, Count, P(Word|Spam) and P(Word|Ham) is the new format of the lists

In [ ]:
p_words_given_spam = {}
p_words_given_ham = {}

p_spam_given_words = {}
p_ham_given_words = {}

p_words = {}

for item in spam_iterator:
    msg_id, metadata_dict = item
    content = metadata_dict['content'].lower()
    p_words_given_spam[msg_id] = 1.0
    p_words[msg_id] = 1.0
    p_spam_given_words[msg_id] = p_spam
    
    for word in content.split():
        for i in range(len(spam_word_count)):
            if spam_word_count[i][0] == word:
                p_words_given_spam[msg_id] = p_words_given_spam[msg_id] * spam_word_count[i][2]
                p_words[msg_id] = p_words[msg_id]* spam_word_count[i][1] / total_words_in_spam
            else:
                continue
    p_spam_given_words[msg_id] = p_words_given_spam[msg_id] * p_spam / p_words[msg_id]
            

for item in ham_iterator:
    msg_id, metadata_dict = item
    content = metadata_dict['content'].lower()
    p_words_given_ham[msg_id] = 1.0
    p_ham_given_words[msg_id] = p_ham
    for word in content.split():
        for i in range(len(ham_word_count)):
            if ham_word_count[i][0] == word:
                p_words_given_ham[msg_id] = p_words_given_ham[msg_id] * ham_word_count[i][2]
            else:
                continue 
    p_ham_given_words[msg_id] = p_words_given_ham[msg_id] * p_ham / p_words[msg_id]


[('!important;', 669, 0.030649588289112534),
 ('{', 456, 0.02090576395242452),
 ('}', 456, 0.02090576395242452),
 ('.', 412, 0.01889295516925892),
 ('font-size:', 207, 0.009515096065873741),
 ('line-height:', 207, 0.009515096065873741),
 ('ie', 171, 0.00786825251601098),
 ('mso', 144, 0.006633119853613907),
 ('width:', 123, 0.005672461116193962),
 ('to', 114, 0.0052607502287282705),
 ('and', 114, 0.0052607502287282705),
 ('.wrapper', 93, 0.004300091491308325),
 ('max-width:', 93, 0.004300091491308325),
 ('the', 82, 0.003796889295516926),
 ('.email-flexible-footer', 66, 0.0030649588289112534),
 ('color:', 57, 0.0026532479414455626),
 ('of', 53, 0.0024702653247941444),
 ('<td', 51, 0.0023787740164684353),
 ('0;', 48, 0.002241537053979872),
 ('20px;', 48, 0.002241537053979872),
 ('<p', 48, 0.002241537053979872),
 ('a', 47, 0.0021957913998170175),
 ('class="size-17"', 45, 0.0021043000914913084),
 ('.column,', 42, 0.001967063129002745),
 ('display:', 42, 0.001967063129002745),
 ('padding-ri